In [ ]:
import os

import pandas as pd
import numpy as np

pd.options.display.max_rows = None

In [ ]:
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

In [ ]:
df = pd.read_excel(os.environ.get("SIAO_FILE_URL"), dtype=str)

### Nettoyage

In [ ]:
df = df.replace([np.nan, ""], None)
df["Code SIRET"] = df["Code SIRET"].replace(r"\.0", "", regex=True).replace(r"\D", "", regex=True).replace("", None).apply(lambda s: s and f"{s:0>14}").replace(r"0{14}", None, regex=True)
df["Code postal"] = df["Code postal"].replace(r"\.0", "", regex=True).replace(r"\D", "", regex=True).replace("", None).apply(lambda s: s and f"{s:0>5}")
df["FINESS"] = df["FINESS"].replace(r"\.0", "", regex=True).replace(r"\D", "", regex=True).replace("", None).apply(lambda s: s and f"{s:0>9}")

### Description générale

* orienté services
* pas de code insee --> à géocoder ou via base sirene géolocalisée
* pas mal de cleanup à faire --> est-ce qu'il est possible d'obtenir le fichier dans un autre format qu'excel ?
* pas de champ date de maj
* pas de champ pour identifier de manière unique et globale les structure sous-jacentes.
* le champ "Nom de la structure" est en fait à mi-chemin entre le nom du service et de la structure.
* pb: lorsque le SIRET est dupliqué, il n'y a pas de moyen fiable pour distinguer les lignes qui sont communes à une structure de celles qui sont des antennes

nb de lignes

In [ ]:
df.shape[0]

nb de lignes avec un numéro FINESS

In [ ]:
df[df["FINESS"].notna()].shape[0]

nb de lignes avec un numéro FINESS ou un siret

In [ ]:
df[df["FINESS"].notna() | df["Code SIRET"].notna()].shape[0]

nb de lignes avec un siret sans FINESS

In [ ]:
df[~df["FINESS"].notna() & df["Code SIRET"].notna()].shape[0]

nb de lignes avec FINESS sans siret

In [ ]:
df[df["FINESS"].notna() & ~df["Code SIRET"].notna()].shape[0]

In [ ]:
df["Code SIRET"].value_counts().apply(lambda c: c == 1).sum()

taux de remplissage des champs

In [ ]:
def compute_field_occupancy_rates(df):
    return ((1 - df.isnull().sum() / df.shape[0]) * 100).sort_values(ascending=False)

compute_field_occupancy_rates(df).to_frame()

### Champs pour extraire un type de structure ?

In [ ]:
df["Type"].value_counts().to_frame()

In [ ]:
df["Catégorie de structure"].value_counts().to_frame()

In [ ]:
structures_df = df[df["Code SIRET"].isna()]

In [ ]:
df[df["Code SIRET"].notna()]["Code SIRET"].nunique()

In [ ]:
categories_flags_structures_df = (
    structures_df["Nom de la structure"]
    .str.lower()
    .apply(
        lambda s: {
            "asso": "association" in s or "asso" in s.split() or "ass" in s.split() or "association" in s.split(), # ASSO
            "ccas": "ccas" in s,  # CCAS
            "mission_locale": "mission" in s and "locale" in s,  # ML
            "cada": "cada" in s.split(),  # CADA
            "chrs": "chrs" in s or "c.h.r.s" in s,  # CHRS
            "chu": "chu" in s.split(),  # CHU
            "cph": "cph" in s.split() or "c.p.h" in s,  # CPH
            "huda": "huda" in s or "h.u.d.a" in s,  # HUDA
            "spip": "spip" in s or "s.p.i.p" in s or "Pénitentiaire" in s,  # SPIP
            "pjj": "pjj" in s or "p.j.j" in s or "protection judiciaire" in s,  # PJJ
            "fjt": "fjt" in s or "f.j.t" in s or ("sidence" in s and "sociale" in s) or s.startswith("rs "),  # RS_FJT
            "udaf": "udaf" in s,  # UDAF
            "plie": "plie" in s.split(),  # PLIE
            "centre_social": "centre" in s and "social" in s,  # CS
            "cias": "cias" in s.split(),  # CIAS
            "cava": "cava" in s.split(),  # CAVA
            "muni": "mairie" in s and "ccas" not in s,  # MUNI
            "caarud": "caarud" in s or "c.a.a.r.u.d" in s,  # CAARUD
            "pe": s.startswith("pole emploi"),  # PE
        },
    )
    .apply(pd.Series)
    .assign(
        na=lambda df: df.apply(
            lambda row: ~row.any(), axis="columns", result_type="expand"
        )
    )
)


In [ ]:
categories_flags_structures_df.sum().sort_values(ascending=False)

In [ ]:
categories_flags_structures_df.iloc[:, categories_flags_structures_df.columns != "na"].any(axis="columns").sum()

In [ ]:
categories_flags_structures_df.sum().sort_values(ascending=False).plot(
    kind="bar", grid=True, rot=35, figsize=(20, 8)
)

### Lien avec la base FINESS

In [ ]:
finess_df = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/3dc9b1d5-0157-440d-a7b5-c894fcfdfd45", dtype=str)
finess_df = finess_df.replace(["", np.nan], None)
finess_df = finess_df[finess_df.siret.notna()]

In [ ]:
merged_df = pd.merge(df, finess_df[["nofinesset", "siret"]], how="left", left_on="FINESS", right_on="nofinesset")

In [ ]:
# lignes avec un siret retrouvé grâce à la base FINESS
merged_df[merged_df.siret.notna() & merged_df["Code SIRET"].isna()].shape[0]

In [ ]:
# structures avec un siret retrouvé grâce à la base FINESS
merged_df[merged_df.siret.notna() & merged_df["Code SIRET"].isna()].siret.nunique()